In [1]:
import pandas as pd
import os
from IPython.display import display
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
import plotly
from plotly.subplots import make_subplots
#sns.set_style("whitegrid")
#sns.set_color_codes("dark")

cols = plotly.colors.DEFAULT_PLOTLY_COLORS
metrics = ["Local_Concordance",
    "Local_Fidelity",
    "Prescriptivity",
    "Conciseness",
    "Robustness"]

In [2]:
if os.path.exists("../../../csvs/"):
    bigDf = []
    for file in os.listdir("../../../csvs/"):
        # Si el tamaño del archivo f"../csvs/{file}" es 0, no lo lee
        if os.stat("../../../csvs/" + file).st_size != 0:
            dataset = pd.read_csv(f"../../../csvs/{file}")
            bigDf.append(dataset)
    bigDf = pd.concat(bigDf)

    bigDf["num_features"] = bigDf["dim"]*bigDf["dim"]
    bigDf["Robustness"] = bigDf["robustness"]
    bigDf["Conciseness"] = bigDf["conciseness"]
    bigDf["Prescriptivity"] = bigDf["prescriptivity"]
    bigDf["Local_Fidelity"] = bigDf["local_fidelity"]
    bigDf["Local_Concordance"] = bigDf["local_concordance"]
    # Eliminar el anterio Experiments.csv
    if os.path.exists("Experiments.csv"):
        os.remove("Experiments.csv")
    bigDf.to_csv("Experiments.csv")

In [3]:
# Show the first 5 rows of the dataframe.
display(bigDf.head())

,conciseness,local_fidelity,local_concordance,prescriptivity,robustness,dataset,sigma,xai_model,dim,max_examples,index,num_features,Robustness,Conciseness,Prescriptivity,Local_Fidelity,Local_Concordance
0,0.724487,0.978930,0.984911,0.996438,0.983105,FashionMNIST,2.0,LIME,7,500,0,49,0.983105,0.724487,0.996438,0.978930,0.984911
1,0.688200,0.976802,0.981163,0.922900,0.983105,FashionMNIST,2.0,LIME,7,500,0,49,0.983105,0.688200,0.922900,0.976802,0.981163
2,0.722753,0.975297,0.984755,0.994791,0.983105,FashionMNIST,2.0,LIME,7,500,0,49,0.983105,0.722753,0.994791,0.975297,0.984755
3,0.716466,0.976538,0.976459,0.991665,0.983105,FashionMNIST,2.0,LIME,7,500,0,49,0.983105,0.716466,0.991665,0.976538,0.976459
4,0.692680,0.977170,0.978329,0.960851,0.983105,FashionMNIST,2.0,LIME,7,500,0,49,0.983105,0.692680,0.960851,0.977170,0.978329


In [8]:
# Wilcoxon test for each metric and each dataset of LIME with sigma = 2.0 and 3.0

# Import the wilcoxon function from scipy.stats
from scipy.stats import wilcoxon

dfLIME_sigma2 = bigDf[(bigDf["xai_model"] == "LIME") & (bigDf["sigma"] == 2.0) & (bigDf["dataset"] != "CIFAR10")]
dfLIME_sigma3 = bigDf[(bigDf["xai_model"] == "LIME") & (bigDf["sigma"] == 3.0) & (bigDf["dataset"] != "CIFAR10")]
wilc_concordance = wilcoxon(dfLIME_sigma2["Local_Concordance"], dfLIME_sigma3["Local_Concordance"])
wilc_fidelity = wilcoxon(dfLIME_sigma2["Local_Fidelity"], dfLIME_sigma3["Local_Fidelity"])
wilc_prescriptivity = wilcoxon(dfLIME_sigma2["Prescriptivity"], dfLIME_sigma3["Prescriptivity"])
wilc_conciseness = wilcoxon(dfLIME_sigma2["Conciseness"], dfLIME_sigma3["Conciseness"])
wilc_robustness = wilcoxon(dfLIME_sigma2["Robustness"], dfLIME_sigma3["Robustness"])



display(wilc_concordance)
display(wilc_fidelity)
display(wilc_prescriptivity)
display(wilc_conciseness)
display(wilc_robustness)

WilcoxonResult(statistic=1715096868.0, pvalue=3.3712410564844056e-12)

WilcoxonResult(statistic=1567716535.0, pvalue=1.098738166611036e-171)

WilcoxonResult(statistic=1740639084.0, pvalue=0.000877981720335106)

WilcoxonResult(statistic=1624087584.0, pvalue=3.2787199389072697e-88)

WilcoxonResult(statistic=1337188360.0, pvalue=0.0)